In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import io
from PIL import Image
import math
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from pyDOE import lhs

viscosity = [0.1, 0.01]
dx =  10
dt =  1000
X = 1
T = 1
x = np.linspace (0, X, dx)
t = np.linspace (0, T, dt)
delta_x = X/(dx-1)
delta_t = T/(dt-1)

class Main_attribute ():
    """
    Class initializate for main attribute which will be using in all mesh
    """
    def __init__ (self):
        self.dx =  dx
        self.dt =  dt

        self.X = X
        self.T = T

        self.x = x
        self.t = t

        self.delta_x = delta_x
        self.delta_t = delta_t

    def condition (self, matrix):
        """
        U(0, x) = sin (pi*x)
        U(t, 0) = 0
        U(t, L) = 0
        """
        matrix [0, :] = np.sin(np.pi * self.x)
        matrix [:, 0] = 0
        matrix [:, -1] = 0
        return matrix
    
    def plot_visualization_1d(self, matrix):
        count = 0
        data = []
        for i in matrix:
            count += 1
            if count % (self.dt/10) == 0:
                time_val = count / self.dt
                for x_val, y_val in zip(self.x, i):
                    data.append({"x": x_val, "y": y_val, "time": time_val})
        df = pd.DataFrame(data)
        plt.figure(figsize=(10, 6))
        sns.lineplot(data=df, x="x", y="y", hue="time", palette="viridis")
        plt.title(self.type)
        plt.xlabel("Координата x")
        plt.ylabel("Скорость u(x,t)")
        plt.legend(title="время", loc='upper right', bbox_to_anchor=(1, 1))
        plt.tight_layout()
        plt.savefig(f"1d_mesh_{self.type}_viscosity_{self.viscosity}.png", format='png') 
        plt.close()
        

    def contour_visualization (self, matrix):
        plt.contourf (self.t, self.x, matrix.T, cmap = 'plasma')
        cbar = plt.colorbar(aspect=10, shrink=0.8)
        cbar.set_label('Value')
        plt.title(self.type)
        plt.ylabel("Координата x")
        plt.xlabel("Время t")
        plt.savefig(f"2d_mesh_{self.type}_viscosity_{self.viscosity}.png", format='png') 
        plt.close()



        
    def CFL_requirments (self):
        if self.delta_t/self.delta_x**2 >= 0.5:
            raise ValueError ("CFL requirments frustrate")
        else:
            print ("Mesh sustainable")

class Straight_mesh (Main_attribute):
    def __init__ (self, viscosity):
        super().__init__()
        self.type = "Явная схема против потока"
        self.main = np.zeros ( (self.dt, self.dx) )
        self.main = self.condition (self.main)
        self.viscosity = viscosity
        self.main = self.explicit_mesh (self.main)
        # self.plot_visualization_1d (self.main)
        # self.contour_visualization (self.main)



    def explicit_mesh (self, matrix):
        self.CFL_requirments ()
        for i in range (self.dt-1):
            for j in range (1, self.dx-1):
                diffusion = self.viscosity * (matrix[i, j+1] - 2*matrix[i, j] + matrix[i, j-1]) / self.delta_x**2
                convection = matrix[i, j] * (matrix[i, j] - matrix[i, j-1]) / self.delta_x
                matrix[i+1, j] = matrix[i, j] + self.delta_t * (diffusion - convection)

        return matrix
    

def create_graph (array_1, array_2, x_array):
        plt.clf ()
        plt.plot (x_array, array_1, label = 'Численные методы, вязкость: 0.1')
        plt.plot (x_array, array_2.cpu().detach().numpy(), label = 'PINN решение, вязкость: 0.1')
        plt.title ("name")
        plt.legend (title="Вязкость", loc='upper right', bbox_to_anchor=(1, 1))
        plt.xlabel("Координата x")
        plt.ylabel("Скорость u(x,t)")
        plt.xlim (0, 1)
        plt.ylim (0, 1)

def create_gif (main_matrix_fdm, model, x_array):
    frames = []
    for i in range (dt):
            if i % (dt//100) == 0:
                time = torch.full (x_array.shape, i)
                predict=model (x_array.reshape(-1, 1), time.reshape(-1, 1))
                create_graph (main_matrix_fdm[i], predict, x_array)
                buf = io.BytesIO()
                plt.savefig(buf, format='png')
                buf.seek(0)
                frames.append(Image.open(buf))
    frames[0].save(f'test.gif',
              save_all=True,
              append_images=frames[1:],
              duration=10,
              loop=0)




In [2]:
if torch.cuda.is_available():
    device = torch.device ("cuda")
else:
    device = torch.device ("cpu")

def init_weight(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform_(m.weight.data)
        torch.nn.init.zeros_(m.bias.data)


class PINN (nn.Module):
    def __init__ (self, n_hidden, n_neurons, activation, verbose = True):
        super().__init__ ()
        self.model = nn.Sequential ()
        self.build_NN (n_hidden, n_neurons, activation)
        self.model.to(device)


        if verbose:
            print (self.model)

        self.model.apply (init_weight)


    def build_NN (self,n_hidden, n_neurons, activation):
        self.model.add_module ('input layer',nn.Linear (2, n_neurons))
        self.model.add_module ('activation func', activation)
        
        for _ in range (n_hidden-1):
            self.model.add_module (f'hidden layer number: {_}', nn.Linear (n_neurons, n_neurons))
            self.model.add_module (f'activation func {_}', activation)

        self.model.add_module ('Output layer',nn.Linear (n_neurons, 1))

    def forward (self, x, t):
        x = torch.concat ([x,t], dim=1).to(device)
        out = self.model (x)
        return out
    

def pde_loss(model, x, t):
    u = model(x, t)
    u_t = torch.autograd.grad(u, t, create_graph=True, grad_outputs=torch.ones_like(t_rod))[0]
    u_x = torch.autograd.grad(u, x, create_graph=True, grad_outputs=torch.ones_like(x_rod))[0]
    u_xx = torch.autograd.grad(u_x, x, create_graph=True, grad_outputs=torch.ones_like(x_rod))[0]
    
    physics_loss = u_t + u * u_x - viscosity * u_xx 
    physics_loss = torch.mean(physics_loss**2)
    return physics_loss

def boundary_loss(model, x, t):

    u = model(x, t)
    loss = torch.mean(torch.square(u))
    return loss

def initial_loss(model, x, t):

    u = model(x, t)
    correct_vals = torch.sin(math.pi * x)
    loss = torch.mean((correct_vals - u)**2)
    return loss

In [3]:
test = Straight_mesh (viscosity[0])


Mesh sustainable


In [4]:
x_plot = torch.linspace(0, 1, 1000).view(-1, 1)
model = PINN (5, 25, nn.Tanh())
model.load_state_dict (torch.load ("model_lhs_0_1.pth"))

Sequential(
  (input layer): Linear(in_features=2, out_features=25, bias=True)
  (activation func): Tanh()
  (hidden layer number: 0): Linear(in_features=25, out_features=25, bias=True)
  (activation func 0): Tanh()
  (hidden layer number: 1): Linear(in_features=25, out_features=25, bias=True)
  (activation func 1): Tanh()
  (hidden layer number: 2): Linear(in_features=25, out_features=25, bias=True)
  (activation func 2): Tanh()
  (hidden layer number: 3): Linear(in_features=25, out_features=25, bias=True)
  (activation func 3): Tanh()
  (Output layer): Linear(in_features=25, out_features=1, bias=True)
)


<All keys matched successfully>

In [6]:

x_plot = torch.linspace(0, 1, 10).view(-1, 1)
t_plot = torch.linspace(0, 1, 1000).view(-1, 1)


def create_graph(array_1, x_array):
    plt.figure()  # Create a new figure
    plt.plot(x_array, array_1, label='Численные методы, вязкость: 0.1')
    plt.title("Динамика скорости u(x,t)")
    plt.legend(title="Вязкость", loc='upper right', bbox_to_anchor=(1, 1))
    plt.xlabel("Координата x")
    plt.ylabel("Скорость u(x,t)")
    plt.xlim(0, 1)
    plt.ylim(0, 1)
    plt.grid(True)  # Optional: Add grid for better readability
    plt.tight_layout()  # Adjust layout to prevent overlap

def create_graph_model(x_array, temp_value_for_graph):
    plt.figure()  # Create a new figure
    x_data = x_array.cpu().detach().squeeze().numpy()
    y_data = temp_value_for_graph.cpu().detach().squeeze().numpy()
    
    plt.plot(x_data, y_data)
    plt.title("PINN нейронная сеть")
    plt.xlabel("Координата x")
    plt.ylabel("Скорость u(x,t)")
    plt.xlim(0, 1)
    plt.ylim(0, 1)
    plt.grid(True)  # Optional: Add grid for better readability
    plt.tight_layout()  # Adjust layout to prevent overlap

def create_gif(x_array, time_array, matrix):
    frames = []
    x_array, time_array = x_array.reshape(-1, 1), time_array.reshape(-1, 1)
    
    for i, time in enumerate(time_array):
        create_graph(matrix[i], x_array)
        
        time_tensor = torch.full_like(x_array, time.item())
        
        # Ensure the model is defined and can handle the input
        try:
            temp_value_for_graph = model(x_array, time_tensor)
            create_graph_model(x_array, temp_value_for_graph)
        except Exception as e:
            print(f"Error during model prediction: {e}")
            continue  # Skip this iteration if there's an error
        
        buf = io.BytesIO()
        plt.savefig(buf, format='png')
        buf.seek(0)
        frames.append(Image.open(buf))
        plt.close()  # Close the figure to free memory

    # Save the frames as a GIF
    frames[0].save('solution_lhs_0_1.gif',
                   save_all=True,
                   append_images=frames[1:],
                   duration=30,
                   loop=0)

# Example usage
# x_plot = torch.linspace(0, 1, 10).view(-1, 1)
# t_plot = torch.linspace(0, 1, 1000).view(-1, 1)
# matrix = ...  # Your data matrix
# create_gif(x_plot, t_plot, matrix)
